In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Shared-Variables-application-1") \
      .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Động lực

- Khi thực hiện các transformation như reduce() hay map() thì spark sẽ thực hiện transform trên các remote node (slave nodes). Các variables sẽ được gửi đi cùng với tasks và không quay lại với PySpark Driver do đó không thể tái sử dụng và chia sẻ các biến này giữa các tasks.

- Do đó PySpark sử dụng các <u>shared variables</u> được cached tại tất cả các nodes:
    + **Broadcast variables (read-only shared variable)**: Việc sử dụng các broadcast varibles sẽ hạn chế việc phải gửi chúng mỗi lần gửi task, do đó tiết kiệm được chi phí communication.
        * One of the best use-case of PySpark RDD Broadcast is to use with lookup data như zip code, state, country lookups e.t.c (các giá trị mặc định mà cần look up thường xuyên)
        * The broadcasted data is cache in serialized format and deserialized before executing each task
    + **Accumulator variables (updatable shared variables)**: là một shared variables khác nhưng cho phép add dữ liệu vào (không chỉ read-only, nhưng chỉ add)
        * **named accumulators**: được hiển thị trên Job UI trong “Accumulator” tab. Trong tab có 2 tables: accumulable (chứa các tên biến và giá trị của chúng) và tasks (chứa value cho từng accumulator được modified bởi task)
        * **unnamed accumulators**: không được lưu trữ, không truyền giá trị string khi khởi tạo 

In [3]:
broadcastVar = spark.sparkContext.broadcast([0, 1, 2, 3])
print(broadcastVar.value)
broadcastVar.unpersist()  # giải phóng khỏi cached ở tất cả executors. Khi cần dùng thì sẽ phải gửi lại cho từng executor
broadcastVar.destroy()    # delete toàn bộ data and metadata. Không thể dùng nữa

[0, 1, 2, 3]


Một vài loại accumulators:
- Long Accumulator: isZero, copy, reset, add, count, sum, avg, merge, value
- Double Accumulator 
- Collection Accumulator: isZero, copyAndReset, copy, reset, add, merge, value

In [9]:
accum = spark.sparkContext.accumulator(value=1)
spark.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).foreach(lambda x: accum.add(x))

print(accum.value)

56


In [10]:
spark.stop()